In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import swifter

# 加载文件
# 这次只分析'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type'
df = pd.read_csv('./20191019_GooglePlayStore.csv', usecols=(0, 1, 2, 3, 4, 5, 6))
# 简单浏览下数据
df.head()
# 查看行列数量
print("数据的形状：", df.shape)
# 查看各个列的非空数据量
print("\n查看各列的空值情况：\n", df.count())
# 有很多缺失值，需要清洗

# App 处理
# 查看有没有重复值
# 也可以：df['App'].value_counts()
print("\n数据的非重复值数量是：", pd.unique(df['App']).size)
print("\n数据的重复值数量是：", df.shape[0] - pd.unique(df['App']).size)
# 有重复值，先不着急删除重复值，为了不把其他列的异常值留下，先处理数值异常的列
# 避免处理异常值后依然又出现新的重复值，从而二次删除重复值

# Category 处理
df['Category'].value_counts(dropna=False)
# # 有一条异常值
# print("\n异常数据是：")
# df[df['Category'] == '1.9']
# # 初步观察，应该是 Category 后的栏下的数据都往前移动一格，导致该数据异常


# Rating 处理，空值用平均值填充
df['Rating'].value_counts(dropna=False)
df['Rating'].fillna(value=df['Rating'].mean(), inplace=True)
# # 有一条值是19的异常记录，和Category的异常是同一条记录

# Reviews清洗
df['Reviews'].value_counts(dropna=False)
df['Reviews'].str.isnumeric().sum()

# 异常值处理
df[~df['Reviews'].str.isnumeric()]
df.drop(index=10472, inplace=True)
df['Reviews'] = df['Reviews'].astype('i8')

# Size的清洗处理
df['Size'].value_counts()
df['Size'] = df['Size'].str.replace('M', 'e+6')
df['Size'] = df['Size'].str.replace('k', 'e+3')

# 尝试转换，结果转换报错，仍有字符串 "Varies with device"
# df['Size'].astype('f8')


# 定义一个字符串判断是否可以转换
def is_convertable(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
# 查看不能转换的字符串分布
temp = df['Size'].swifter.apply(is_convertable)
df['Size'][~temp].value_counts()

# 使用 0 替换剩下的字符串
df['Size'] = df['Size'].str.replace('Varies with device', '0')

# 看是否还有没转换的字符串
temp = df['Size'].apply(is_convertable)
df['Size'][~temp].value_counts()

# 转换类型
# e+5 这种格式使用 astype 直接转为 int 有问题，如果想转成 int ，可以先转成 f8 ，再转 i8
# df['Size'] = df['Size'].astype('f8').astype('i8')
df['Size'] = df['Size'].astype('f8')

# 将 Size 为 0 的填充为平均数
df['Size'].replace(0, df['Size'].mean(), inplace=True)
df.describe()

# Installs数据清洗
# 先查看分布
df['Installs'].value_counts()
# 分布比较少，直接替换
df['Installs'] = df['Installs'].str.replace('+', '')
df['Installs'] = df['Installs'].str.replace(',', '')
# 转换
df['Installs'] = df['Installs'].astype('i8')
df.describe()

# Type 处理
# df.info()查看到有 nan 值，这里需要 dropna 参数
df['Type'].value_counts(dropna=False)
df[df['Type'].isnull()]
# 删除这条数据
df.drop(index=9148, inplace=True)

# 删除 App 重复的行
df.drop_duplicates('App', inplace=True)


# 数据清洗完毕
# 整体情况
df.describe()

# 分Category的数据
# 分类的个数
df.Category.unique().size

# 每个分类的App数量，排序，可以得出哪些分类的app最受开发者欢迎
df.groupby('Category').count().sort_values('App', ascending=False)

# 分类的安装量排序：娱乐社交类最被用户所需要
df.groupby('Category').mean().sort_values('Installs', ascending=False)

# 分类的评论数据：社交游戏视频评论多
df.groupby('Category').mean().sort_values('Reviews', ascending=False)

# 分类的打分数据，和其他数据不太一致，需要进一步分析
df.groupby('Category').mean().sort_values('Rating', ascending=False)

# 分Type数据
# 免费占比大，付费占比小，免费仍然是主流
df.groupby('Type').count()

# 只有两个类型，且数据量差别很大
df.groupby('Type').sum().sort_values('Installs', ascending=False)

# Category和Type一起分析
df.groupby(['Type', 'Category']).mean().sort_values('Reviews', ascending=False)

# # 评论安装比
# 收费的app评论比率更高
g = df.groupby(['Type', 'Category']).mean()
(g['Reviews'] / g['Installs']).sort_values(ascending=False)


# 相关性：评论数和安装数强相关，其他的连0.1都不到，可以认为是不相关的（0.5以上可以认为是相关的，0.3以上可以认为是弱相关）
df.corr()

数据的形状： (10841, 7)

查看各列的空值情况：
 App         10841
Category    10841
Rating       9367
Reviews     10841
Size        10841
Installs    10841
Type        10840
dtype: int64

数据的非重复值数量是： 9660

数据的重复值数量是： 1181


,Rating,Reviews,Size,Installs
Rating,1.000000,0.054278,0.052600,0.039174
Reviews,0.054278,1.000000,0.080578,0.625164
Size,0.052600,0.080578,1.000000,0.050675
Installs,0.039174,0.625164,0.050675,1.000000
